<a href="https://colab.research.google.com/github/Xueying-Ya/NLP-Com-ling/blob/master/NeuralNetworkTrain_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Model for polarity**

In [ ]:
from keras.layers.wrappers import Bidirectional

Using TensorFlow backend.


In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from IPython.display import Image, display_png
from gensim.models import word2vec, KeyedVectors
from keras.models import Sequential
from keras.layers import Input, Embedding, Dense, Dropout, Flatten, LSTM,Bidirectional,GlobalAveragePooling1D
from keras.utils import to_categorical, plot_model
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
!pip install contractions
import pandas as pd
import contractions
import nltk
from nltk.tokenize import word_tokenize
nltk.download('all')
import spacy
nlp = spacy.load("en_core_web_sm")

     |████████████████████████████████| 317kB 3.7MB/s 
     |████████████████████████████████| 245kB 16.4MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81700 sha256=8f8457af4e8d45042c4bd45512f4bffe6e13eb3fc8bcb07e62bceebc5303c802
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloa

In [ ]:
!gdown --id 1oVHLl-SsEQ9q2PVDVRAwvYv91ks3lyur

Downloading...
From: https://drive.google.com/uc?id=1oVHLl-SsEQ9q2PVDVRAwvYv91ks3lyur
To: /content/GoogleNews-vectors-negative300.bin
3.64GB [01:04, 56.4MB/s]


In [ ]:
data = pd.read_csv('lab4_train.csv')

In [ ]:
conflict_word_list = ['but','although','though','despie','spite','nevertheless','however','nonetheless','notwithstanding','contray','contrast']

def tokenize_filter(string_text):
  filter_words = []
  contracts = contractions.fix(string_text) #เปลี่ยนคำย่อ สมมติว่า didn't เป็น did not 
  token_list = word_tokenize(contracts)
  pos_sentences=nltk.pos_tag(token_list)
  for tup in pos_sentences:
      if tup[1].startswith(('J','N','R')) or tup[0] in conflict_word_list  : #เอา adj noun adv ไว้
        filter_words.append(tup[0])
  return filter_words

def lemma(string_text):
  lemma_word = []
  contracts = contractions.fix(string_text) #เปลี่ยนคำย่อ สมมติว่า didn't เป็น did not 
  doc = nlp(contracts)
  for token in doc:
      lemma_word.append(token.lemma_)
  return lemma_word

def lemma_filter(string_text):
  lemma_filter_list = []
  contracts = contractions.fix(string_text) #เปลี่ยนคำย่อ สมมติว่า didn't เป็น did not 
  doc = nlp(contracts)
  lemma = [token.lemma_ for token in doc]
  pos_sentences=nltk.pos_tag(lemma)
  for tup in pos_sentences:
      if tup[1].startswith(('J','N','R','V')) or tup[0] in conflict_word_list  : #เอา adj noun adv ไว้
        lemma_filter_list.append(tup[0])
  return lemma_filter_list

data['tokens'] = data['text'].apply(lemma_filter)

In [ ]:
data['tokens'][2000]

['-PRON-', 'love', '-PRON-']

In [ ]:
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:

vector_dim = (len(model['I']))

In [ ]:
vocab_in_data = set()
for tokens in data['tokens']:
    for token in tokens:
        vocab_in_data.add(token)
print(len(vocab_in_data))

3546


In [ ]:
vector_dim = model.vector_size

In [ ]:
vector_dim

300

In [ ]:
print(vector_dim)

300


In [ ]:
max_len = 30

In [ ]:
vocab_size = len(vocab_in_data) + 1

In [ ]:
word_to_index = {'PADDING':0}

# make weight matrix of word embedding, vocab size + 1 (for padding)
embedding_matrix = np.zeros((vocab_size, vector_dim), dtype="float32")
embedding_matrix[0] = np.zeros(vector_dim)

for i, word in enumerate(vocab_in_data & set(model.vocab)):
    word_to_index[word] = i+1
    embedding_matrix[i+1] = model[word] 

In [ ]:
import numpy as np
train, dev = np.split(data, [int(0.8 * len(data))])

In [ ]:
def w2i(tokens):
    return [word_to_index.get(word, 0) for word in tokens[:max_len]]

# apply functions & convert to np.array
input_train = np.array(train['tokens'].apply(w2i).tolist())
input_dev = np.array(dev['tokens'].apply(w2i).tolist())

# input : zero padding
train_x = pad_sequences(input_train, max_len, padding='post')
dev_x = pad_sequences(input_dev, max_len, padding='post')

In [ ]:
def relabel(label):
    return {'negative':0, 'positive':1, 'conflict':2, 'neutral':3}[label]
def reaspect(aspect):
    return {'service':0, 'food':1, 'anecdotes/miscellaneous':2, 'price':3,'ambience':4}[aspect]

In [ ]:
# apply functions & convert to np.array
label_train = np.array(train['polarity'].apply(relabel).tolist())
label_dev = np.array(dev['polarity'].apply(relabel).tolist())

aspect_train = np.array(train['aspectCategory'].apply(reaspect).tolist())
aspect_dev = np.array(dev['aspectCategory'].apply(reaspect).tolist())

# label : one-hot vector
train_label = to_categorical(label_train, num_classes=4)
dev_label = to_categorical(label_dev, num_classes=4)

train_aspect = to_categorical(aspect_train, num_classes=5)
dev_aspect = to_categorical(aspect_dev, num_classes=5)

In [ ]:
# check the shape
print('input train:', train_x.shape)
print('input dev:', dev_x.shape)
print('label train:', train_y.shape)
print('label dev:',dev_y.shape)

input train: (2524, 30)
input dev: (632, 30)


NameError: ignored

In [ ]:
Polarity
# instantiation
model = Sequential()

# add embedding layer
model.add(Embedding(input_dim=vocab_size,
                    input_length=max_len,
                    output_dim=vector_dim, 
                    weights=[embedding_matrix], 
                    mask_zero=True,
                    trainable=True))

# average
model.add(LSTM(200,recurrent_dropout = 0.2))

# add hidden layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.50))



# add output layer
model.add(Dense(4, activation='softmax'))

# compile model
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

In [ ]:
#Polarity
# instantiation
model = Sequential()

# add embedding layer
model.add(Embedding(input_dim=vocab_size,
                    input_length=max_len,
                    output_dim=vector_dim, 
                    weights=[embedding_matrix], 
                    mask_zero=True,
                    trainable=True))

# average
model.add(Bidirectional(LSTM(200,dropout = 0.2,recurrent_dropout = 0.2)))

# add hidden layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.20))
model.add(Dense(50, activation='relu'))


# add output layer
model.add(Dense(4, activation='softmax'))

# compile model
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

In [ ]:
plot_model(model, show_shapes=True,to_file='model.png')
display_png(Image('model.png'))

In [ ]:
# train
history = model.fit(train_x, train_label, batch_size=128, epochs=12, validation_data=(dev_x, dev_label))

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='best')
plt.show()

In [ ]:
!gdown --id 1YtAHCzeZUXGZQ9cimdkkUq4lUk3ZH-I_

Downloading...
From: https://drive.google.com/uc?id=1YtAHCzeZUXGZQ9cimdkkUq4lUk3ZH-I_
To: /content/evaluate.py
100% 7.03k/7.03k [00:00<00:00, 12.7MB/s]


In [ ]:
def index_to_label(polarity_index):
    return {0: 'negative', 1: 'positive', 2: 'conflict', 3: 'neutral'}[polarity_index]
def index_to_aspect(aspect_index):
    return {0:'service', 1:'food', 2:'anecdotes/miscellaneous', 3:'price',4:'ambience'}[aspect_index]


In [ ]:
prediction = model.predict(dev_x)
prediction = [index_to_label(np.argmax(i)) for i in prediction]


In [ ]:
newdf = pd.DataFrame()
newdf['id'] = dev['id']
newdf['aspectCategory'] = dev['aspectCategory'] #not predicted yet 
newdf['polarity'] = prediction
newdf.to_csv('pred.csv', index=None)
newdf

In [ ]:
!python3 evaluate.py lab4_train.csv pred.csv 

In [ ]:
#Aspect
# instantiation
model = Sequential()

# add embedding layer
model.add(Embedding(input_dim=vocab_size,
                    input_length=max_len,
                    output_dim=vector_dim, 
                    weights=[embedding_matrix], 
                    mask_zero=True,
                    trainable=True))

# average
model.add(Bidirectional(LSTM(200,recurrent_dropout = 0.2)))

# add hidden layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.50))


# add output layer
model.add(Dense(5, activation='softmax'))

# compile model
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

In [ ]:
plot_model(model, show_shapes=True,to_file='model.png')
display_png(Image('model.png'))

In [ ]:
history = model.fit(train_x, train_aspect, batch_size=128, epochs=15, validation_data=(dev_x, dev_aspect))

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='best')
plt.show()

In [ ]:
prediction = model.predict(dev_x)
prediction = [index_to_aspect(np.argmax(i)) for i in prediction]

In [ ]:
aspect_file = pd.DataFrame()
aspect_file['id'] = dev['id']
aspect_file['polarity'] = dev['polarity'] #not predicted yet 
aspect_file['aspectCategory'] = prediction
aspect_file.to_csv('asp_pred.csv', index=None)
aspect_file

In [ ]:
!python3 evaluate.py lab4_train.csv asp_pred.csv